# Классификация комментариев

**Задача**:<br>
Разработать модель, которая будет определять токсичен текст комментария или нет. Значение метрики качества F1 модели на тестовой выборке должна быть не менее 0.75

**Дано**:<br>
Набор комментариев с разметкой о токсичности.

**План решения**:
1. Загрузить данные
2. Подготовить корпус текстов: очистить тексты с помощью регулярных выражений и лемматизировать
4. Разбить корпус текстов на обучающую и тестову выборки
3. Рассчитать значения TF-IDF для обучающей и тестовой выборок
5. Обучить разные модели и сравнить их метрики F1, выбрать лучшую модель
6. Проверить значение метрики F1 лучшей модели на тестовой выборке

## Подготовка

In [1]:
#Импортируем библиотеки и модули
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords, wordnet
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
#Установм лемматизатор Spacy
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.9 MB 2.0 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
#Загрузим модуль для обработки текста
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [4]:
#Создадим датасет
data = pd.read_csv('/datasets/toxic_comments.csv')

In [5]:
#Выведем несколько случайных строк для проверки
data.sample(10)

,Unnamed: 0,text,toxic
152615,152772,"""\n\n Portals \n\nNice work critiquing the Eco...",0
41005,41055,LUCKY where is my arbratary file?,0
66452,66519,Please report me. There must be mediators out ...,0
131629,131765,United Kingdom isnt a country. so where does t...,0
72613,72684,How is this not worth mentioning?,0
138048,138195,"""\n\n You are mistaken (again). I am happy to ...",0
68140,68208,it's unlike anything else in the capital district,0
146409,146565,God no. There are hundreds of jaggeds. There a...,0
127150,127281,"""\nHi! Great if you improve the English langua...",0
18969,18988,"Roster\nupdated for 2008 missing numbers, nee...",0


In [6]:
#Выведем информацию о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [7]:
#Проверим на дубликаты
print(data.duplicated().sum())

0


In [8]:
#Создадим корпус текстов
corpus = data['text'].values

In [10]:
#Создадим функцию для лемматизации текста
def lemm_text(text):
    clear_text = " ".join(re.sub(r'[^a-zA-Z ]', ' ', text).split()) #очищаем текст с помощью рег. выражения
    doc = nlp(clear_text)
    lemmatized_text = " ".join([token.lemma_ for token in doc]) 
    return lemmatized_text

In [11]:
%%time

#Созадим столбец с очищенным и лемматизированным текстом
data['lemm_text'] = data['text'].apply(lemm_text)

CPU times: user 16min 37s, sys: 5.97 s, total: 16min 43s
Wall time: 16min 45s


In [12]:
#Выведем датасет для проверки
data.sample(10)

,Unnamed: 0,text,toxic,lemm_text
141408,141560,"""\n\nI'm not withdrawing anything. Jimbo Wale...",0,I m not withdraw anything Jimbo Wales be a wha...
55154,55215,CardinalDan FUCK YOU FAGGOT NIGGER CUNT LICKIN...,1,CardinalDan fuck you FAGGOT nigger CUNT lickin...
114879,114978,"Exactly, the BBC's Blue Peter/The One Show Spe...",0,exactly the BBC s Blue Peter the one Show spec...
104090,104187,Factual correction to,0,factual correction to
148655,148811,British Isles British Islands,0,british Isles british island
42830,42880,PROBLEM \n\nit needs a picture,0,problem it need a picture
112620,112718,". If no reliable text can be found, an image ...",0,if no reliable text can be find an image of an...
113059,113157,""":: I'll try looking for sources about impact ...",0,I ll try look for source about impact and stuf...
41295,41345,"""\n\n You are canvassing re: Gatineau deletion...",0,you be canvass re Gatineau deletion you should...
115007,115106,Engines \n\nthe Saturn AL-41F will be one of t...,0,engine the Saturn AL F will be one of the fina...


## Обучение

Разделим данные на обучающую и тестовую выборку

In [13]:
#Выделим признаки и целевой признак
X = data['lemm_text']
y = data['toxic']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345)

Подготовим обучающую выборку

In [15]:
corpus_train = X_train.values

In [16]:
#Создадим список стоп-слов
stopwords = set(nltk_stopwords.words('english'))

In [17]:
#Cоздадим счётчик, указав в нём стоп-слова
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [18]:
#Рассчитаем TF-IDF для корпуса текстов обучающей выборки
tf_idf = count_tf_idf.fit_transform(corpus_train)

In [19]:
#Назовём полученные значения TF-IDF обучающей выборкой
X_train = tf_idf.copy()

In [20]:
#Удалим ненужные переменные
del tf_idf

Подготовим тестовую выборку

In [21]:
corpus_test = X_test.values

In [22]:
#Рассчитаем TF-IDF для корпуса текстов тестовой выборки
tf_idf = count_tf_idf.transform(corpus_test)

In [24]:
#Назовём полученные значения TF-IDF тестовой выборкой
X_test = tf_idf.copy()

In [25]:
#Удалим ненужные переменные
del tf_idf, data, X, y

Изучим баланс классов в обучающей выборке

In [27]:
y_train.value_counts(normalize=True)

0    0.898367
1    0.101633
Name: toxic, dtype: float64

В обучающей выборке обнаружен дисбаланс классов. При обучении моделей будем учитывать дисбаланс

Обучим модели и подберём гиперпараметры. В гиперпараметрах моделей укажем сбалансированные веса классов.
- LogisticRegression
- RandomForestClassifier
- CatBoostClassifier

**LogisticRegression**

In [33]:
model_lr = LogisticRegression(max_iter=1000, class_weight='balanced')

In [29]:
%%time

#Найдём значение метрики f1 с помощью кросс-валидации
final_scores = []
scores = cross_val_score(model_lr, X_train, y_train, scoring='f1', cv=5)
final_scores.append(scores.max())

final_f1 = pd.Series(final_scores).max()
print('Лучшее значение метрики F1:', final_f1)

Лучшее значение метрики F1: 0.7553903345724907
CPU times: user 2min 3s, sys: 2min 49s, total: 4min 53s
Wall time: 4min 53s


Метрика F1 модели удовляетворяет условиям задачи

In [34]:
%%time

#Найдём время обучения
model_lr.fit(X_train, y_train)

CPU times: user 31.6 s, sys: 39.7 s, total: 1min 11s
Wall time: 1min 11s


LogisticRegression(class_weight='balanced', max_iter=1000)

**RandomForestClassifier**

In [30]:
%%time

# Объявляем модель
model_rf = RandomForestClassifier(class_weight='balanced', random_state=12345)

# словарь с гиперпараметрами и значениями, которые хотим перебрать
# Указал небольшой диапазон, т.к. модель очень долго обучается
param_grid_rf = {'n_estimators': np.arange(20, 101, 20), 'max_depth': np.arange(10, 31, 10),}

gs_rf = GridSearchCV(
    model_rf, 
    param_grid=param_grid_rf, 
    scoring='f1', 
    n_jobs=-1
)

gs_rf.fit(X_train, y_train)

# лучшее значение F1 на кросс-валидации
print(f'best_score: {gs_rf.best_score_ }')

# лучшие гиперпараметры
print(f'best_params: {gs_rf.best_params_}')

best_score: 0.41989632220315604
best_params: {'max_depth': 30, 'n_estimators': 100}
CPU times: user 35min 50s, sys: 11.1 s, total: 36min 1s
Wall time: 36min 39s


Метрика F1 модели не удовляетворяет условиям задачи

**CatBoostClassifier**

In [32]:
%%time

#Объявим модель со встроенным методом кодирования категориальных признаков
model_cb = CatBoostClassifier(verbose=True, random_state=12345)

# словарь с гиперпараметрами и значениями, которые хотим перебрать. С параметром max_depth 8 и более падает ядро.
param_grid_cb = {'n_estimators': [100, 200, 300], 'max_depth': [4, 6]}

gs_cb = GridSearchCV(
    model_cb, 
    param_grid=param_grid_cb, 
    scoring='f1', 
    n_jobs=-1
)

gs_cb.fit(X_train, y_train)

# лучшее значение F1 на кросс-валидации
print(f'Лучшее значение метрики F1: {gs_cb.best_score_ }')

# лучшие гиперпараметры
print(f'Гиперпараметы: {gs_cb.best_params_}')

Learning rate set to 0.5
0:	learn: 0.3529442	total: 984ms	remaining: 1m 37s
1:	learn: 0.2707286	total: 1.87s	remaining: 1m 31s
2:	learn: 0.2424229	total: 2.8s	remaining: 1m 30s
3:	learn: 0.2301230	total: 3.63s	remaining: 1m 27s
4:	learn: 0.2191698	total: 4.51s	remaining: 1m 25s
5:	learn: 0.2136323	total: 5.36s	remaining: 1m 23s
6:	learn: 0.2076097	total: 6.18s	remaining: 1m 22s
7:	learn: 0.2012577	total: 7.08s	remaining: 1m 21s
8:	learn: 0.1978270	total: 8.04s	remaining: 1m 21s
9:	learn: 0.1949544	total: 8.91s	remaining: 1m 20s
10:	learn: 0.1922238	total: 9.81s	remaining: 1m 19s
11:	learn: 0.1896494	total: 10.7s	remaining: 1m 18s
12:	learn: 0.1865614	total: 11.5s	remaining: 1m 17s
13:	learn: 0.1839332	total: 12.4s	remaining: 1m 16s
14:	learn: 0.1818336	total: 13.3s	remaining: 1m 15s
15:	learn: 0.1794070	total: 14.1s	remaining: 1m 14s
16:	learn: 0.1767177	total: 15s	remaining: 1m 13s
17:	learn: 0.1752414	total: 15.8s	remaining: 1m 12s
18:	learn: 0.1734550	total: 16.7s	remaining: 1m 11s


Метрика F1 модели не удовляетворяет условиям задачи

По условиям задачи необходимое качество метрики F1 > 0.75 показала модель **Логистическая регрессия**. Проверим модель на тестовй выборке: 

In [38]:
print('F1-мера модели на тестовой выборке:', f1_score(y_test, model_lr.predict(X_test)))

F1-мера модели на тестовой выборке: 0.7534763393338364


Результат модели **Логистическая регрессия** на тестовой выборке удовляетворяет условиям задачи.

## Выводы

По условиям задачи необходимо было разработать модель, которая будет определять, токсичен текст комментария или нет. Значение метрики F1 на тестовой выборке должно составлть не менее 0.75.<br><br>
В ходе работы тексты комментариев были подготовлены: очищены с помощью регулярных выражений и лемматизированы. Для корпусов текстов были найдены значения TF-IDF, которые применялись для обучения моделей.<br><br>
С помощью кросс-валидации были проверены следующие модели с разными гиперпараметрами:
- LogisticRegression
- RandomForestlassifier
- CatBoostRegressor
<br><br>

Максимальное значение метрики F1 равное 0.755 показала модель **Логистическая регрессия** (LogisticRegression). На тестовой выборке модель также показала удовлетворительное значение метрики F1, равное 0.753.